## Climate Data


This is the climate data.  Source for the climate data will be posted here

In [16]:
# retrieve stored variables 
%store -r habitat_data_dir teton_gdf rmnp_gdf aoi_gdf

In [1]:
#import libraries 

import os
import pathlib
from glob import glob

import earthaccess
import geopandas as gpd
import xrspatial
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # CRSs

import xarray as xr

In [4]:
# Set required data directories 
climate_data_dir = os.path.join(habitat_data_dir,'climate')
os.makedirs(climate_data_dir, exist_ok=True)

In [ ]:
#format URL 

maca_url = (
  'http://thredds.northwestknowledge.net:8080'
  '/thredds/dodsC/MACAV2/BNU-ESM'
  '/macav2metdata_'
  #Varabable tasmin, daily average min temperature 
  'tasmin_BNU-ESM_r1i1p1'
  #climate scenario 
  '_rcp85'
  #years, in 5 years chunks
  '_2096_2099'
  #monthly or daily 
  '_CONUS_monthly'
  #file format, need to open with xrray 
  '.nc'
)

maca_ds = xr.open_dataset(maca_url)
maca_ds

<xarray.Dataset> Size: 156MB
Dimensions:          (lat: 585, crs: 1, lon: 1386, time: 48)
Coordinates:
  * lat              (lat) float64 5kB 25.06 25.1 25.15 ... 49.31 49.35 49.4
  * crs              (crs) int32 4B 1
  * lon              (lon) float64 11kB 235.2 235.3 235.3 ... 292.9 292.9 292.9
  * time             (time) object 384B 2096-01-15 00:00:00 ... 2099-12-15 00...
Data variables:
    air_temperature  (time, lat, lon) float32 156MB ...
Attributes: (12/46)
    description:                     Multivariate Adaptive Constructed Analog...
    id:                              MACAv2-METDATA
    naming_authority:                edu.uidaho.reacch
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    Metadata_Link:                   
    cdm_data_type:                   FLOAT
    ...                              ...
    contributor_role:                Postdoctoral Fellow
    publisher_name:                  REACCH
    publisher_email:                 reacch@uidaho.edu
    publisher_url:                   http://www.reacchpna.org/
    license:                         Creative Commons CC0 1.0 Universal Dedic...
    coordinate_system:               WGS84,EPSG:4326

In [ ]:
#correct datum values in data set

In [ ]:

maca_ds.rio.clip_box(*teton_gdf.to_crs(total_bounds)

NoDataInBounds: No data found in bounds. Data variable: air_temperature